In [36]:
# Importing automation tools
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# Importing data manipulation tools
import pandas as pd
import numpy as np


# Importing web scraping tools
from bs4 import BeautifulSoup
import requests


# Importing database & database engine
import sqlite3
import sqlalchemy

# Importing miscellaneous
from datetime import datetime
import pause
import schedule
import time
from schedule import every, repeat, run_pending
import time


# Setting up database engine and database connection
engine = sqlalchemy.create_engine('sqlite:///QQQ_information.db')
connection = sqlite3.connect('QQQ_information.db')
cursor = connection.cursor()

# Setting up pathways and initializing webdriver 
PATH = '/Users/omkar/Documents/chromedriver'
ser = Service(PATH)
op = webdriver.ChromeOptions()
s = webdriver.Chrome(service=ser, options=op)

# Opening desired webpage and saving url + session id
s.get('https://client.schwab.com/Areas/Trade/Options/Chains/Index.aspx#symbol/QQQ;strategy/greeks;expiration/custom|2022++02+02&QQQ:25&QQQ;strike/all;expandcollapse/3|2022++01+31&QQQ++02+02&QQQ:04&QQQ:25&QQQ')
url = s.command_executor._url
session_id = s.session_id 

driver = webdriver.Remote(command_executor=url,desired_capabilities={})
driver.close()   
driver.session_id = session_id
s.maximize_window()


In [37]:
# Creating automation function 
def get_qqq_info():
    global s
    global url
    global session_id
    url = s.command_executor._url
    session_id = s.session_id 
    
    driver = webdriver.Remote(command_executor=url,desired_capabilities={})
    driver.close()   
    driver.session_id = session_id
    
    # Automated webpage navigations to select by Greek values + refreshing page
    drop_down = s.find_element(By.ID, 'btnStrategy')
    drop_down.click()
    time.sleep(2)
    greeks = s.find_element(By.ID, 'greeks')
    greeks.click()
    time.sleep(2)
    refresh = s.find_element(By.ID, "refresh-all-quotes")
    refresh.click()
    time.sleep(5)
    
    # Finding + saving desired webpage dataframes
    last_trade_time = s.find_element(By.XPATH, '//*[@id="chains-symbol-entry"]/div[2]/div[2]/div[1]/div[3]')
    day1 = s.find_element(By.XPATH, '//*[@id="exp-0-header"]/tr[2]/td[1]/span')
    day2 = s.find_element(By.XPATH, '//*[@id="exp-1-header"]/tr[2]/td[1]')
    day3 = s.find_element(By.XPATH, '//*[@id="exp-2-header"]/tr[2]/td[1]/span')
    day_rows1 = s.find_element(By.XPATH, '//*[@id="exp-0-div"]/div[1]/span[2]/b[1]')
    day_rows2 = s.find_element(By.XPATH, '//*[@id="exp-1-div"]/div[1]/span[2]/b[1]')
    day_rows3 = s.find_element(By.XPATH, '//*[@id="exp-2-div"]/div[1]/span[2]/b[1]')

    # Creating list and saving dataframe rows 
    elements = []
    elements = s.find_elements(By.CLASS_NAME, 'strike-row')

    # Calculating + adding number of all the rows in the dataframes 
    Time = []
    total_row_values = int(day_rows1.text) + int(day_rows2.text) + int(day_rows3.text)
    i = 0 
    while i != total_row_values:
        Time.append(last_trade_time.text[11:24])
        i = i + 1
    
    #Adding certain values of dataframe row to list 
    Day = []
    i = 0 
    while i != int(day_rows1.text):
        Day.append(day1.text[4:16])
        i = i +1
  
    i = 0 
    while i != int(day_rows2.text):
        Day.append(day2.text[4:16])
        i = i +1
    
    i = 0 
    while i != int(day_rows3.text):
        Day.append(day3.text[4:16])
        i = i +1


    # Seperating various Greek column values to their respective variables 
    Vega = []
    Theta = []
    Gamma = []
    Delta = []
    IV = []
    Bid = []
    Ask = []
    Strike = []
    Bid2 = []
    Ask2 = []
    IV2 = []
    Delta2 = []
    Gamma2  = []
    Theta2 = []
    Vega2 = []

    for element in elements:
        Vega.append(element.text.split(' ')[1])
        Theta.append(element.text.split(' ')[2])
        Gamma.append(element.text.split(' ')[3])
        Delta.append(element.text.split(' ')[4])
        IV.append(element.text.split(' ')[5])
        Bid.append(element.text.split(' ')[6])
        Ask.append(element.text.split(' ')[7])
        Strike.append(element.text.split(' ')[8])
        Bid2.append(element.text.split(' ')[9])
        Ask2.append(element.text.split(' ')[10])
        IV2.append(element.text.split(' ')[11])
        Delta2.append(element.text.split(' ')[12])
        Gamma2.append(element.text.split(' ')[13])
        Theta2.append(element.text.split(' ')[14])
        Vega2.append(element.text.split(' ')[15])

    url = s.command_executor._url
    session_id = s.session_id 

    driver = webdriver.Remote(command_executor=url,desired_capabilities={})
    driver.close()   
    driver.session_id = session_id

    # Creating dictionary of above variables to add as data to Pandas dataframe
    d = {'Day': Day, 'Time': Time, 'Vega' : Vega, 'Theta' : Theta, 'Gamma' : Gamma, 'Delta' : Delta, "IV" : IV, "Bid" : Bid, "Ask" : Ask, "Strike" : Strike, "Bid2" : Bid2, "Ask2" : Ask2, 'IV2' : IV2, "Delta2" : Delta2, 'Gamma2' : Gamma2, "Theta2" : Theta2, 'Vega2' : Vega2}
    data_table_greeks = pd.DataFrame(data = d)
    
    # Saving Pandas database to sql database 
    data_table_greeks.to_sql('QQQ_Greeks', engine, if_exists = 'append', index=False)


    # More automated navigation to select Calls/Puts values + refreshing page
    url = s.command_executor._url
    session_id = s.session_id 
    
    driver = webdriver.Remote(command_executor=url,desired_capabilities={})
    driver.close()   
    driver.session_id = session_id
    time.sleep(2)
    drop_down = s.find_element(By.ID, 'btnStrategy')
    drop_down.click()
    time.sleep(2)
    calls_puts = s.find_element(By.ID, 'calls-and-puts')
    calls_puts.click()
    time.sleep(5)
    refresh = s.find_element(By.ID, "refresh-all-quotes")
    refresh.click()
    time.sleep(6)
    
    # Finding + saving desired webpage dataframes
    last_trade_time = s.find_element(By.XPATH, '//*[@id="chains-symbol-entry"]/div[2]/div[2]/div[1]/div[3]')
    day1 = s.find_element(By.XPATH, '//*[@id="exp-0-header"]/tr[2]/td[1]/span')
    day2 = s.find_element(By.XPATH, '//*[@id="exp-1-header"]/tr[2]/td[1]')
    day3 = s.find_element(By.XPATH, '//*[@id="exp-2-header"]/tr[2]/td[1]/span')

    day_rows1 = s.find_element(By.XPATH, '//*[@id="exp-0-div"]/div[1]/span[2]/b[1]')

    day_rows2 = s.find_element(By.XPATH, '//*[@id="exp-1-div"]/div[1]/span[2]/b[1]')

    day_rows3 = s.find_element(By.XPATH, '//*[@id="exp-2-div"]/div[1]/span[2]/b[1]')
    
    elements2 = []
    elements2 = s.find_elements(By.CLASS_NAME, 'strike-row')

    #Adding certain values of dataframe row to list 
    Time = []
    total_row_values = int(day_rows1.text) + int(day_rows2.text) + int(day_rows3.text)
    i = 0 
    while i != total_row_values:
        Time.append(last_trade_time.text[11:24])
        i = i + 1
    
    Day = []
    i = 0 
    while i != int(day_rows1.text):
        Day.append(day1.text[4:16])
        i = i +1
  
    i = 0 
    while i != int(day_rows2.text):
        Day.append(day2.text[4:16])
        i = i +1
    
    i = 0 
    while i != int(day_rows3.text):
        Day.append(day3.text[4:16])
        i = i +1
        
    # Seperating various Call/Put column values to their respective variables 
    Volume = []
    OI = []
    Last = []
    Change = []
    Bid = []
    Ask = []
    Strike = []
    Bid2 = []
    Ask2 = []
    Last2 = []
    Change2 = []
    Volume2 = []
    OI2 = []
    

    for element in elements2:
        Volume.append(element.text.split(' ')[2])
        OI.append(element.text.split(' ')[3])
        Last.append(element.text.split(' ')[4])
        Change.append(element.text.split(' ')[5])
        Bid.append(element.text.split(' ')[6])
        Ask.append(element.text.split(' ')[7])
        Strike.append(element.text.split(' ')[8])
        Bid2.append(element.text.split(' ')[9])
        Ask2.append(element.text.split(' ')[10])
        Last2.append(element.text.split(' ')[11])
        Change2.append(element.text.split(' ')[12])
        Volume2.append(element.text.split(' ')[13])
        OI2.append(element.text.split(' ')[14])

    # Creating dictionary of above variables to add as data to Pandas dataframe
    d2 = {'Day' : Day, 'Time' : Time, 'Volume' : Volume, 'OI' : OI, 'Last' : Last, 'Change' : Change, "Bid" : Bid, "Ask" : Ask, "Strike" : Strike, "Bid2" : Bid2, "Ask2" : Ask2, "Last2" : Last2, 'Change2' : Change2, "Volume2" : Volume2, 'OI2' : OI2}
    data_table_cp = pd.DataFrame(data = d2)
    
    # Saving Pandas database to sql database 
    data_table_cp.to_sql('QQQ_c/p', engine, if_exists = 'append', index=False)
    print('ran')
    



In [ ]:
schedule.clear()
#                   *ALL TIME INTERVALS ARE WRITTEN IN MILITARY FORMAT*
#                            ENTER START VALUE HERE:
#                    Year  Month Date Hour Minute Second
pause.until(datetime(2022,   2,   28,  9,   26,    00   ))
get_qqq_info()
#                            ENTER END VALUE HERE:
#              run code until... (Time)
schedule.every(10).minutes.until("10:25").do(get_qqq_info)
while True:
    schedule.run_pending()
    time.sleep(1) 